In [ ]:
import os
import numpy as np
import matplotlib
import torch
# !pip install mat73
# import mat73
import matplotlib.pyplot as plt
import csv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split,TensorDataset
from torchvision import transforms, utils
import time
import pandas as pd
import scipy.io
import sklearn.metrics
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from torchvision.transforms import ToTensor

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
np.random.seed(1)

from scipy import signal

from sklearn.metrics import confusion_matrix

import seaborn as sn
import pandas as pd

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import LeaveOneGroupOut






In [ ]:
from math import sqrt
import torch
import torch.nn.functional as F
from torch import nn
!pip install einops
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class LSA(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        self.heads = heads
        self.temperature = nn.Parameter(torch.log(torch.tensor(dim_head ** -0.5)))

        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.temperature.exp()

        mask = torch.eye(dots.shape[-1], device = dots.device, dtype = torch.bool)
        mask_value = -torch.finfo(dots.dtype).max
        dots = dots.masked_fill(mask, mask_value)

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, LSA(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

class SPT(nn.Module):
    def __init__(self, *, dim, patch_size, channels = 3):
        super().__init__()
        patch_dim = patch_size * patch_size * 5 * channels

        self.to_patch_tokens = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_size, p2 = patch_size),
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim)
        )

    def forward(self, x):
        shifts = ((1, -1, 0, 0), (-1, 1, 0, 0), (0, 0, 1, -1), (0, 0, -1, 1))
        shifted_x = list(map(lambda shift: F.pad(x, shift), shifts))
        x_with_shifts = torch.cat((x, *shifted_x), dim = 1)
        return self.to_patch_tokens(x_with_shifts)

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = SPT(dim = dim, patch_size = patch_size, channels = channels)

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)

In [ ]:


def plotfigFullClass(cm,acc,tle,ExpVer):

    parent_dir = r"/content/drive/My Drive/Colab Notebooks/RFMG/data/"+ExpVer+"/result/"
    path1 = os.path.join(parent_dir, 'cm_fig')
    os.makedirs(path1, exist_ok = True) 
    path1 = os.path.join(parent_dir, 'npyFile')
    os.makedirs(path1, exist_ok = True) 



    name1=["R", "G","G*2","P1","P1*2","P2","P2*2","P23","P23*2","P4","P4*2","sG","sF","sP1","sP2","sP23","sP4","U","U*2","D","D*2","sU","sD"]
    name2=name1
    w=20 #fig size 1      change when label num change
    h=22   #fig size 2
    
    # true_num=0
    # for i in range (len(cm)):
    #   true_num=true_num+ cm[i,i]
    # acc=true_num/np.sum(cm)   
    
    df_cm = pd.DataFrame(cm, index=name1, columns=name2)
    plt.rcParams["font.family"] = "Times New Roman"
    plt.figure()
    sn.set(font_scale=1.2)
    mask = np.zeros_like(df_cm)
    mask[np.where(cm==0)] = True
    
    sn.heatmap(df_cm, cmap="YlGnBu",fmt=".2f",vmin=0, vmax=1.0,annot=True,mask=mask,square=True,cbar=False,annot_kws={'fontsize':15})
 
    s01=' Acc={n:.3f}'.format(n=acc)
    #plt.text(a,b,s01,fontsize=size)
    
    tleSave=tle
    plt.title(tle+s01)

    figure = plt.gcf()
    figure.set_size_inches(w, h)
    
    plt.savefig(parent_dir+r"cm_fig/"+tleSave+"CM.png", dpi=300)




In [ ]:
in_ch=48
channel=np.linspace(0,47,48).astype(int) 
#self_channel=[0,1,10,11,20,21,30,31]
from google.colab import drive
drive.mount('/content/drive')


# in_ch=2
# channel=[48,49]
num_class=23
k=7


file_name_data=r"/content/drive/My Drive/Colab Notebooks/RFMG/data/"+'2_5'+"/data_all_multi6_filt_0.1_5.npz"
data=np.load(file_name_data)
feature_2d_stft=data['feature_2d_stft']
feature_2d_stft2=data['feature_2d_stft2']
feature_2d_cwt=data['feature_2d_cwt']
feature_2d_cwt2=data['feature_2d_cwt2']
feature_2d_cwt3=data['feature_2d_cwt3']

label_all=data['label_all']
caseNum_all=data['caseNum_all']


opt1={'image_size':56,'patch_size':7}

opt_list=[opt1]









Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# in_ch: CNN input channel number 
# num_class: classification number  8,5,3 
# opt CV method. groupKfold: kfold, all cases not been seen in training set  
# Kfold : randomly kfold. label and cases equally distributed
# group: leave one case out 

par_vit={'dim':512,'depth':6,'heads':16,'mlp_dim':64,'dropout':0.1,'emb_dropout':0.1,\
         'n_epoch':25,'learning_rate':1e-4}; 
par_vit={'dim':512,'depth':6,'heads':16,'mlp_dim':64,'dropout':0.1,'emb_dropout':0.1,\
         'n_epoch':20,'learning_rate':1e-4};   
parameter_vit=np.array(list(par_vit.items()));
from google.colab import drive
drive.mount('/content/drive')
X_all=feature_2d_stft
y_all=label_all
opt=opt1

if __name__ == '__main__':   
      
    cm_all=[]
    acc_all=[]
    y_test_all=[]
    y_p_all=[]
    test_ind_all=[]

    start_timeall=time.time()
    training_time=[]
 


 

    X_train=X_all[:-200,:,:] 
    X_test=X_all[-200:,:,:]
    y_train=y_all[:-200] 
    y_test=y_all[-200:]
    
    

        

  

    batchsize_train = 16
    train_loader = DataLoader(TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train)), batch_size=batchsize_train,shuffle=True)
    batchsize_test = 200
    test_loader = DataLoader(TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test)), batch_size=batchsize_test, shuffle=False)
            
    
    n_epoch=par_vit['n_epoch']
    #n_epoch=25
    learning_rate = par_vit['learning_rate']  #学习的速率 越小越精细
    momentum = 0.1
    random_seed=1
    torch.backends.cudnn.enabled = False
    torch.manual_seed(random_seed)     #设定随机数种子为固定值
    
    train_loss_epoch = []
    test_acc_epoch =[]
    total_acc_epoch = []
    #epoch_range = np.arange(25,225,25)
    epoch_range = np.array([2])

    
    
    network = ViT(
    image_size = opt['image_size'],
    patch_size = opt['patch_size'],
    num_classes =num_class,
    dim = par_vit['dim'],
    depth = par_vit['depth'],
    heads = par_vit['heads'],
    mlp_dim = par_vit['mlp_dim'],
    dropout = par_vit['dropout'],
    emb_dropout = par_vit['emb_dropout'],
    channels = in_ch
    ).cuda()
    #training
    optimizer = optim.Adam(network.parameters(), lr=learning_rate)
    
    
    
    #network.train()
    Training_Loss = []
    Test_Loss = []
    
    criterion = nn.CrossEntropyLoss()
    for epoch in range(n_epoch):   # loop over the dataset multiple times
        train_loss = 0
        for X, Y in train_loader:
            
            X = X.float().cuda()  
            Y = Y.long().view(-1, ).cuda() 
            current_batchsize = X.shape[0]
           
            optimizer.zero_grad()
            output = network(X)
            loss = criterion(output,Y)
            train_loss = train_loss + loss.item()
            loss.backward()                     #calculate the gradient decent
            optimizer.step()                    #update the weight
            

        test_loss = 0
        correct = 0
        total = 0
        test_y= []
        test_y_p = []
        
        # sens = 0    # sensitivity = TP/(TP + FN)
        # prec = 0    # precision = TP/(TP + FP)
        with torch.no_grad():        
            # X = X.view(-1,X.shape[1],X.shape[2])
            # X = X.float()         
            for X, Y in test_loader:
                X = torch.Tensor(X).cuda()
                Y = torch.Tensor(Y).long().view(-1, ).cuda()
                
                images, labels = X, Y
                # calculate outputs by running images through the network
                start_time=time.time()
                outputs = network(images)
                
                
                loss = criterion(outputs,Y)
                test_loss = test_loss + loss.item()

                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(outputs.data, 1)
                training_time.append(time.time()-start_time)
                total += labels.size(0)
           
                
                for i in range(len(labels)):
                  test_y.append(labels[i])
                  test_y_p.append(predicted[i])
                  

        Training_Loss.append(train_loss/len(train_loader.dataset))
        Test_Loss.append(test_loss/len(test_loader.dataset))
        # if epoch%(n_epoch-1)==0:
        #     print(train_loss/len(train_loader.dataset))
        #     print(test_loss/len(test_loader.dataset))
  
    train_loss_epoch.append(Training_Loss[-1])
    test_y_p=torch.FloatTensor(test_y_p)
    test_y=torch.FloatTensor(test_y)
  
    y_p=np.array(test_y_p.cpu())
    y=np.array(test_y.cpu())
    

    cm=confusion_matrix(y, y_p)
    acc=accuracy_score(y, y_p)
    
    cm_all.append(cm)
    acc_all.append(acc)
    for i in range(len(y_p)):
        y_p_all.append(y_p[i])
        y_test_all.append(y_test[i])
    print(acc) 



acc_all=np.array(acc_all)
cm_all=np.array(cm_all)
y_test_all=np.array(y_test_all)
y_p_all=np.array(y_p_all)
y=np.stack((y_test_all, y_p_all))

test_ind_all=np.array(test_ind_all)

cm=confusion_matrix(y_test_all, y_p_all)
cm_norm=confusion_matrix(y_test_all, y_p_all,normalize='true')

acc=accuracy_score(y_test_all, y_p_all)

x_timeall=time.time()-start_timeall
print(np.mean(np.array(training_time)))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
0.295
0.1266635537147522


In [ ]:
!nvidia-smi

Mon Mar 21 15:29:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    33W / 250W |   4593MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------